# Preprocessing

To run locally, simply change the paths used to open the files to the corresponding paths and run all cells.

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers, optimizers

## Read training data and labels

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !ls /content/drive/My\ Drive/

ModuleNotFoundError: No module named 'google'

In [ ]:
# --- Read training data ---
f = open('/content/drive/MyDrive/test/data/train-images-idx3-ubyte/train-images.idx3-ubyte', 'rb')

# Read header (16 bytes)
f.read(16)

# The rest of the data are the images.
# The intensity on the grey scale of each pixel is stored in one byte
# The images are of size 28 x 28 which is why the take up 784 adjacent bytes
training_images = []
while True:
    # Leemos la siguiente imagen
    image = f.read(784)
    if len(image) != 784: # Si no queda imagen parar
        break
    else: # Conversión de binario a entero
        training_images.append([x for x in image])

# Convert to numpy array
training_images = np.array(training_images)

# --- Read training labels ---
f = open('/content/drive/MyDrive/test/data/train-labels-idx1-ubyte/train-labels.idx1-ubyte', 'rb')

# Read header (8 bytes)
f.read(8)

# The rest of the data are the labels. Each label is one byte
labels_wrong_format = np.array([x for x in f.read()])

## Read test data

In [4]:
# Read in evaluation labels
# Ejemplo de lectura de MNIST (imágenes). Leer archivo en binario
f = open('/content/drive/MyDrive/test/data/t10k-images-idx3-ubyte/t10k-images.idx3-ubyte', 'rb')

# Read header (16 bytes)
f.read(16)

# El resto de datos son las imágenes.
# La intensidad en escala de grises para cada píxel se guarda en un byte
# Las imágenes son de 28 * 28 por lo que ocupan 784 bytes adyacentes
images_testing = []
while True:
    # Leemos la siguiente imagen
    image_test = f.read(784)
    if len(image_test) != 784: # Si no queda imagen parar
        break
    else: # Conversión de binario a entero
        images_testing.append([x for x in image_test])

# Conversión a array de NumPy
images_testing = np.array(images_testing).reshape(-1, 28, 28, 1)

# Lectura de etiquetas
f = open('/content/drive/MyDrive/test/data/t10k-labels-idx1-ubyte/t10k-labels.idx1-ubyte', 'rb')

# Descartar cabecera (8 bytes)
f.read(8)

# El resto de datos son las etiquetas. Cada etiqueta es un byte
labels_testing = np.array([x for x in f.read()])

## Bring data into correct format

In [5]:

training_images = training_images / 255.0
training_images = training_images.reshape(-1, 28, 28, 1)
labels_training = tf.keras.utils.to_categorical(labels_wrong_format)

images_testing = images_testing / 255.0
images_testing = images_testing.reshape(-1, 28, 28, 1)
labels_testing = tf.keras.utils.to_categorical(labels_testing)

# Create model

## Model configuration

In [6]:
BATCH_SIZE = 64

# Data augmentation to help prevent overfitting see, here:
# https://stackoverflow.com/questions/70080062/how-to-correctly-use-imagedatagenerator-in-keras
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False
)
datagen.fit(training_images)
datagen_generator = datagen.flow(training_images, labels_training, batch_size=BATCH_SIZE)

In [7]:
# Define a learning rate schedule
def lr_schedule(epoch):
    initial_lr = 0.0005
    if epoch < 5:
        return initial_lr * (epoch + 1) / 5
    return initial_lr * 0.5 ** ((epoch - 5) // 5)

In [8]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True, verbose=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_callbacks = [early_stopping, lr_scheduler]

In [ ]:
def create_model():
    model = tf.keras.models.Sequential()

    # Block 1 - input_shape=(BATCH_SIZE, 1, 28, 28) output_shape=(BATCH_SIZE, 32, 28, 28)
    model.add(layers.Conv2D(32, (5, 5), padding="same", input_shape=(28, 28, 1)))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.2))

    # Block 2 - input_shape=(BATCH_SIZE, 32, 28, 28) output_shape=(BATCH_SIZE, 64, 14, 14)
    model.add(layers.Conv2D(64, (5, 5), padding="same"))
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))

    # Block 3 - input_shape=(BATCH_SIZE, 64, 14, 14) output_shape=(BATCH_SIZE, 96, 14, 14)
    model.add(layers.Conv2D(96, (5, 5), padding="same"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.2))

    # Block 4 - input_shape=(BATCH_SIZE, 96, 14, 14) output_shape=(BATCH_SIZE, 128, 7, 7)
    model.add(layers.Conv2D(128, (5, 5), padding="same"))
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))

    # Block 5 - input_shape=(BATCH_SIZE, 128, 7, 7) output_shape=(BATCH_SIZE, 128 * 7 * 7)
    model.add(layers.Flatten())

    # Block 5 - input_shape=(BATCH_SIZE, 128 * 7 * 7) output_shape=(BATCH_SIZE, 10)
    model.add(layers.Dense(10))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
EPOCHS = 100

model.fit(
    datagen_generator,
    epochs=EPOCHS,
    callbacks=model_callbacks
)

# Total training time around 3022 seconds

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 32ms/step - accuracy: 0.4644 - loss: 1.6745 - learning_rate: 1.0000e-04
Epoch 2/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 27s 28ms/step - accuracy: 0.8982 - loss: 0.5894 - learning_rate: 2.0000e-04
Epoch 3/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9574 - loss: 0.2407 - learning_rate: 3.0000e-04
Epoch 4/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9722 - loss: 0.1259 - learning_rate: 4.0000e-04
Epoch 5/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 27ms/step - accuracy: 0.9757 - loss: 0.0929 - learning_rate: 5.0000e-04
Epoch 6/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9823 - loss: 0.0652 - learning_rate: 5.0000e-04
Epoch 7/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9827 - loss: 0.0582 - learning_rate: 5.0000e-04
Epoch 8/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9839 - loss: 0.0555 - learning_rate: 5.0000e-04
Epoch 9/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.9857 - 

In [11]:
# # Optionally load model
# model = tf.keras.models.load_model('final_model.keras')

In [12]:
model.evaluate(images_testing, labels_testing)

# Predict the labels for the test images
predictions = model.predict(images_testing)

# Convert predictions to label indices
predicted_labels = np.argmax(predictions, axis=1)

# Convert true labels from one-hot encoding to label indices
true_labels = np.argmax(labels_testing, axis=1)

# Compare predicted labels with true labels
correct_predictions = np.sum(predicted_labels == true_labels)

print(f"Number of correctly recognized images: {correct_predictions} out of {len(images_testing)}")
error = (len(images_testing) - correct_predictions) / len(images_testing)
print(f"The error rate is {error * 100}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9971 - loss: 0.0109
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Number of correctly recognized images: 9975 out of 10000
The error rate is 0.25%


In [13]:
model.save('model.keras')

In [1]:
# Output result in desired format
result = ""

for index in range(len(images_testing)):
  prediction = model.predict(images_testing[index].reshape(1, 28, 28, 1))
  predicted_label = np.argmax(prediction)
  result += str(predicted_label)

with open('result.txt', 'w') as f:
  f.write(result)

NameError: name 'images_testing' is not defined